# **Mineduc Data Cleansing** ✨

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Vivamus sit amet consectetur diam. Fusce auctor facilisis nulla et pharetra. Maecenas at lacus eu nisl scelerisque pretium. Nulla at tempor lectus. Mauris eu magna lectus. Nam posuere sapien quis tortor tempor, quis maximus mi pulvinar. Praesent eget tincidunt massa. Sed vulputate nisl et imperdiet gravida. Proin at enim vitae lectus maximus convallis. Proin aliquet sapien eget mauris vestibulum maximus. Nulla facilisi. 

**Authors:**
- [Daniel Valdez](https://github.com/Danval-003)
- [Emilio Solano](https://github.com/emiliosolanoo21)
- [Adrian Flores](https://github.com/adrianRFlores)
- [Andrea Ramírez](https://github.com/Andrea-gt)

***

## **(1) Import Libraries** ⬇️

In [323]:
# Data manipulation and visualization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from fuzzywuzzy import process, fuzz
import difflib
import re
from unidecode import unidecode

# Standard libraries
import warnings
warnings.filterwarnings('ignore')

# ===== ===== Reproducibility Seed ===== =====
# Set a fixed seed for the random number generator for reproducibility
random_state = 42

# Set matplotlib inline
%matplotlib inline

# Set default figure size
plt.rcParams['figure.figsize'] = (6, 4)

# Define custom color palette
palette = sns.color_palette("viridis", 12)

# Set the style of seaborn
sns.set(style="whitegrid")

## **(2)Data Upload** 📄

In [324]:
df = pd.read_csv('data/establecimientos.csv')
df.head(2)

,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
0,16-01-0138-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO COBAN,KM.2 SALIDA A SAN JUAN CHAMELCO ZONA 8,77945104,MERCEDES JOSEFINA TORRES GALVEZ,GUSTAVO ADOLFO SIERRA POP,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
1,16-01-0139-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO PARTICULAR MIXTO VERAPAZ,KM 209.5 ENTRADA A LA CIUDAD,77367402,MERCEDES JOSEFINA TORRES GALVEZ,GILMA DOLORES GUAY PAZ DE LEAL,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ


## **(2) Exploratory Analysis** 🔎

**Observaciones 💡 -->**

> - Este conjunto de datos se enfoca en la recopilación de información sobre establecimientos educativos en el país Guatemala. Los datos fueron obtenidos de la página oficial del [Ministerio de Educación](https://www.mineduc.gob.gt/BUSCAESTABLECIMIENTO_GE/). Incluye más de 9000 registros, aunque esto se detallará más adelante, que abarcan detalles de relevancia para el análisis y gestión de la educación en el país.

### **(1) Descripción General de los Datos**

In [325]:
# Print the number of records in the DataFrame
print("The given dataset has", df.shape[0], "registers and", df.shape[1], "columns.")

The given dataset has 9354 registers and 17 columns.


**Observaciones 💡 -->**

> - El conjunto de datos original contiene 9354 registros y 17 columnas, lo que sugiere que posee una dimensión relativamente grande. Cada uno de los 9354 registros representa una observación única, mientras que las 17 columnas corresponden a diferentes características o variables medidas para cada observación.

In [326]:
# Basic information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9354 entries, 0 to 9353
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CODIGO           9331 non-null   object
 1   DISTRITO         9123 non-null   object
 2   DEPARTAMENTO     9331 non-null   object
 3   MUNICIPIO        9331 non-null   object
 4   ESTABLECIMIENTO  9331 non-null   object
 5   DIRECCION        9280 non-null   object
 6   TELEFONO         8792 non-null   object
 7   SUPERVISOR       9122 non-null   object
 8   DIRECTOR         8434 non-null   object
 9   NIVEL            9331 non-null   object
 10  SECTOR           9331 non-null   object
 11  AREA             9331 non-null   object
 12  STATUS           9331 non-null   object
 13  MODALIDAD        9331 non-null   object
 14  JORNADA          9331 non-null   object
 15  PLAN             9331 non-null   object
 16  DEPARTAMENTAL    9331 non-null   object
dtypes: object(17)
memory usage: 1.2+ 

**Observaciones 💡 -->**

> - En el análisis preliminar del conjunto de datos, se observa que varias columnas, aunque almacenadas como objetos, contienen datos que podrían beneficiarse de una conversión a tipos más apropiados aunque no es escencial. Por ejemplo, las columnas CODIGO, TELEFONO y DISTRITO, a pesar de ser objetos, ***podrían*** ser tratadas como datos categóricos o numéricos si se requiere un análisis más detallado. Sin embargo, estas son decisiones que se tomarán en el futuro y por ahora es una simple observación.

### **(2) Clasificación de las Variables**

| Nombre           | Descripción                                                                                       | Clasificación                                                                                   |
|------------------|---------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------|
| Código           | Identifica de manera única a cada establecimiento.                                                | Variable cualitativa descriptiva                                                               |
| Establecimiento  | Nombre del establecimiento educativo.                                                             | Variable cualitativa descriptiva                                                               |
| Dirección        | Ubicación física del establecimiento.                                                             | Variable cualitativa descriptiva                                                               |
| Teléfono         | Número de teléfono del establecimiento.                                                           | Variable  cualitativa descriptiva                                                                 |
| Director         | Nombre del director del establecimiento.                                                          | Variable cualitativa descriptiva                                                               |
| Distrito         | Clasifica el establecimiento dentro de un distrito específico.                                     | Variable cualitativa categórica                                                                |
| Departamento     | Indica el departamento donde se encuentra cada establecimiento.                                    | Variable cualitativa categórica                                                                |
| Municipio        | Define el municipio donde se localiza cada establecimiento.                                        | Variable cualitativa categórica                                                                |
| Nivel            | Nivel educativo ofrecido por el establecimiento (ej. diversificado, primaria, etc.).               | Variable cualitativa categórica                                                                |
| Sector           | Tipo de sector educativo (ej. público, privado).                                                  | Variable cualitativa categórica                                                                |
| Área             | Área geográfica del establecimiento (ej. urbana, rural).                                          | Variable cualitativa categórica                                                                |
| Estado           | Estado actual del establecimiento (ej. abierto, cerrado).                                          | Variable cualitativa categórica                                                                |
| Modalidad        | Modalidad de enseñanza ofrecida (ej. monolingüe, bilingüe).                                        | Variable cualitativa categórica                                                                |
| Jornada          | Tipo de jornada educativa (ej. matutina, vespertina).                                              | Variable cualitativa categórica                                                                |
| Plan             | Plan educativo implementado (ej. diario, semanal).                                                | Variable cualitativa categórica                                                                |
| Departamental    | Departamento administrativo al que pertenece el establecimiento.                                   | Variable cualitativa categórica                                                                |
| Supervisor       | Nombre del supervisor del establecimiento.                                                        | Variable cualitativa categórica                                                                |

**Observaciones 💡 -->**

> - El conjunto de datos posee alrededor de 4 variables cualitativas descriptivas.
> - El resto de variables, 12 de ellas, son variables cualitaticas categóricas.

### **(3) Exploración y Limpieza Inicial de los Datos**

In [327]:
# Transpose the first 3 rows of the DataFrame
# to display column names as rows for better readability.
df.head(3).T

,0,1,2
CODIGO,16-01-0138-46,16-01-0139-46,16-01-0140-46
DISTRITO,16-031,16-031,16-031
DEPARTAMENTO,ALTA VERAPAZ,ALTA VERAPAZ,ALTA VERAPAZ
MUNICIPIO,COBAN,COBAN,COBAN
ESTABLECIMIENTO,COLEGIO COBAN,COLEGIO PARTICULAR MIXTO VERAPAZ,"COLEGIO ""LA INMACULADA"""
DIRECCION,KM.2 SALIDA A SAN JUAN CHAMELCO ZONA 8,KM 209.5 ENTRADA A LA CIUDAD,7A. AVENIDA 11-109 ZONA 6
TELEFONO,77945104,77367402,78232301
SUPERVISOR,MERCEDES JOSEFINA TORRES GALVEZ,MERCEDES JOSEFINA TORRES GALVEZ,MERCEDES JOSEFINA TORRES GALVEZ
DIRECTOR,GUSTAVO ADOLFO SIERRA POP,GILMA DOLORES GUAY PAZ DE LEAL,VIRGINIA SOLANO SERRANO
NIVEL,DIVERSIFICADO,DIVERSIFICADO,DIVERSIFICADO


**🖊 Modificación de Etiquetas de Variables -->**

> - Para facilitar la comprensión y el manejo del conjunto de datos, se procederá a modificar los nombres de las variables. Este cambio permitirá una organización más clara y una interpretación más precisa de la información.

In [328]:
rename_col = {
    'CODIGO': 'code',
    'DISTRITO': 'district',
    'DEPARTAMENTO': 'department',
    'MUNICIPIO': 'municipality',
    'ESTABLECIMIENTO': 'school_name',
    'DIRECCION': 'address',
    'TELEFONO': 'phone',
    'SUPERVISOR': 'supervisor',
    'DIRECTOR': 'director',
    'NIVEL': 'education_level',
    'SECTOR': 'sector',
    'AREA': 'area',
    'STATUS': 'status',
    'MODALIDAD': 'teaching_mode',
    'JORNADA': 'schedule',
    'PLAN': 'plan',
    'DEPARTAMENTAL': 'departmental'
}

In [329]:
# Use a pandas function to rename the current function
df = df.rename(columns = rename_col)
df.head(2)

,code,district,department,municipality,school_name,address,phone,supervisor,director,education_level,sector,area,status,teaching_mode,schedule,plan,departmental
0,16-01-0138-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO COBAN,KM.2 SALIDA A SAN JUAN CHAMELCO ZONA 8,77945104,MERCEDES JOSEFINA TORRES GALVEZ,GUSTAVO ADOLFO SIERRA POP,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
1,16-01-0139-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO PARTICULAR MIXTO VERAPAZ,KM 209.5 ENTRADA A LA CIUDAD,77367402,MERCEDES JOSEFINA TORRES GALVEZ,GILMA DOLORES GUAY PAZ DE LEAL,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ


#### **(1) Frecuencias y Conteo de los Datos**

In [330]:
# Create a list with descriptive columns
descriptive = ["code", "school_name", "address", "director", "number"]

# Create a list with categorical columns
categorical = ["district", "department", "municipality", "education_level", "sector", "area", "status", "teaching_mode", "schedule", "plan", "departmental", "supervisor"]

In [331]:
group_size = 6  # Len of groups to show
num_groups = len(categorical) // group_size + (len(categorical) % group_size > 0) # Number of groups

# Iterate over the groups and print the description
for i in range(num_groups):
    start = i * group_size
    end = (i + 1) * group_size
    group_cols = categorical[start:end]    
    
    # Use .describe() to get the description of the columns
    description = df[group_cols].describe()
    styler = description.style.set_table_styles([
        {'selector': 'th', 'props': [('font-weight', 'bold')]}
    ])
    # Display the description
    display(styler)

,district,department,municipality,education_level,sector,area
count,9123,9331,9331,9331,9331,9331
unique,690,23,349,1,4,3
top,01-403,CIUDAD CAPITAL,ZONA 1,DIVERSIFICADO,PRIVADO,URBANA
freq,268,1564,633,9331,7956,7606


,status,teaching_mode,schedule,plan,departmental,supervisor
count,9331,9331,9331,9331,9331,9122
unique,4,2,6,13,26,659
top,ABIERTA,MONOLINGUE,DOBLE,DIARIO(REGULAR),GUATEMALA NORTE,CARLOS HUMBERTO GONZALEZ DE LEON
freq,6545,9039,3037,5721,1050,333


**🏅 Top 10 Valores Por Columna -->**

In [332]:
# See what are the 10 most frequent values for each of the dataframe columns
for column in df.columns:
    frequency_values = df[column].value_counts().head(10)
    print("Top 10 most frequent values for column '{}':".format(column))
    for index, (value, frequency) in enumerate(frequency_values.items(), start=1):
        print("{:<5} {:<30} {:<10}".format(index, value, frequency))
    print("\n===============================================")

Top 10 most frequent values for column 'code':
1     16-01-0138-46                  1         
2     17-01-4075-46                  1         
3     17-03-0095-46                  1         
4     17-03-0098-46                  1         
5     17-03-0099-46                  1         
6     17-03-0101-46                  1         
7     17-03-0102-46                  1         
8     17-03-0104-46                  1         
9     17-03-0111-46                  1         
10    17-03-0113-46                  1         

Top 10 most frequent values for column 'district':
1     01-403                         268       
2     11-017                         176       
3     05-033                         167       
4     01-411                         167       
5     18-008                         130       
6     05-007                         104       
7     01-641                         103       
8     18-039                         102       
9     10-019                         

**Observaciones 💡 -->**

1. Es importante señalar que la columna `code` contiene únicamente valores únicos para cada entrada, lo que sugiere que no aporta información significativa al conjunto de datos y podría considerarse prescindible para los análisis posteriores.

2. Por otro lado, la columna `district` presenta múltiples entradas repetidas, siendo 01-403 la más frecuente en el conjunto de datos. Esta repetición podría ser útil para identificar patrones en la distribución de la data, aunque se requiere un análisis más profundo para determinar su relevancia.

3. En cuanto a la columna `municipality`, se ha observado que contiene entradas inconsistentes. En esta columna se registran, en su mayoría, los nombres de los municipios de los departamentos, pero también aparecen nombres de zonas específicas de la ciudad capital, lo que indica una falta de un formato de entrada adecuado. En relación con esto último, es importante tener en cuenta que los municipios de Guatemala no incluyen sus zonas, por lo que la presencia de estas entradas no solo es inconsistente, sino también incorrecta en el contexto de los datos. Todo esto ees algo a tomar en consideración durante el proceso de limpieza.

4. Un aspecto interesante relacionado con la integridad de los datos se observa en la columna `director`. En esta columna, se encuentran 83 valores representados como "---", 28 como "----", 27 como "--" y 14 como "SIN DATO". Esta variedad en la representación de los valores nulos es significativa y debe ser considerada cuidadosamente en el análisis de datos faltantes. Es crucial realizar las modificaciones necesarias para clasificar correctamente estos valores nulos, ya que su manejo inadecuado podría afectar las conclusiones extraídas del conjunto de datos. Vemos este mismo problema en la columna `area`, donde los valores nulos tienen OTRO tipo de codificación, siendo esta "SIN ESPECIFICAR".

5. Por otro lado, la columna `education_level` presenta una notable falta de diversidad en sus entradas, ya que se limita únicamente al valor "DIVERSIFICADO". Esta homogeneidad plantea la pregunta sobre la utilidad de esta columna en la data, ya que no proporciona información escencial que permita realizar comparaciones o análisis significativos.

6. La columna `plan` presenta una distribución notablemente desigual en sus valores, con "DIARIO(REGULAR)" como el más frecuente y otros planes mostrando significativamente menos entradas.  Se considera que se debería realizar una limpieza de datos en esta columna, lo que incluye verificar la consistencia en la nomenclatura de los planes, unificar categorías similares, manejar posibles datos faltantes y considerar la eliminación o agrupación de planes con baja frecuencia que no aporten valor significativo.

7. La columna `departamental` incluye nombres de departamentos del país, pero también presenta regiones específicas que son inconsistentes en su nomenclatura. Dada esta situación, se sugiere agrupar ciertas entradas para proporcionar mayor coherencia y utilidad a la columna. En particular, sería recomendable unificar las diversas regiones de la ciudad capital bajo la etiqueta "Guatemala" y asegurarse de que no existan otras entradas que estén separadas por regiones, lo que podría llevar a confusiones en el análisis.

#### **(2) Análisis de Valores Duplicados**

In [333]:
# Check duplicate rows in dataset
df = df.drop_duplicates()

In [334]:
# Print the number of records in the DataFrame
print("The given dataset has", df.shape[0], "registers and", df.shape[1], "columns.")

The given dataset has 9332 registers and 17 columns.


**Observaciones 💡 -->**

> - Se identificaron 22 entradas duplicadas en el conjunto de datos, lo que representa aproximadamente el 0.5% del total.
> - Este problema ya fue abordado durante la etapa de limpieza preliminar.

## **(4) Data Cleansing 🫧**

### **(1) Errores Estructurales**

<div class="alert alert-block alert-info">
<b>Nota:</b> Los errores estructurales surgen al recolectar o trasladar datos, evidenciándose en nombres inusuales, faltas de ortografía o uso incorrecto de mayúsculas, llevando así a clasificaciones erróneas.
</div>

#### **(1) Columna "department"**

En primer lugar, es fundamental abordar el tema de la capitalización de los nombres de los departamentos en Guatemala. En el dataset original, se ha encontrado que los nombres de los departamentos están mayormente en minúsculas, lo que representa un evidente error. La correcta capitalización de los nombres no solo es importante desde el punto de vista estético, sino que también garantiza la integridad y precisión de la información, evitando confusiones que podrían surgir en el manejo y la interpretación de los datos.

In [335]:
# Capitalizing the values of a single column
df['department'] = df['department'].str.title()

A continuación, se llevará a cabo un análisis más detallado de los valores contenidos en la columna de departamentos. En primer lugar, se convertirá esta columna en una lista para facilitar el análisis. A partir de esta lista, se extraerán los valores únicos, para tratar de identificar posibles errores relacionados con la acentuación, así como otros tipos de inconsistencias que puedan existir.

In [336]:
# Converting the column to a list
column_to_list = df['department'].unique().tolist()

# Convert all elements to strings to avoid TypeError
column_to_list_str = [str(department) for department in column_to_list]

# Pretty print the list and its length
print("Unique Departments:")
print(", ".join(column_to_list_str))
print(f"\nTotal number of unique departments: {len(column_to_list_str)}")

Unique Departments:
Alta Verapaz, nan, Baja Verapaz, Chimaltenango, Chiquimula, Ciudad Capital, El Progreso, Escuintla, Guatemala, Huehuetenango, Izabal, Jalapa, Jutiapa, Peten, Quetzaltenango, Quiche, Retalhuleu, Sacatepequez, San Marcos, Santa Rosa, Solola, Suchitepequez, Totonicapan, Zacapa

Total number of unique departments: 24


**Observaciones 💡 -->**

Vemos que existen un punto importante a resolver.
1. **Inconsistencias en la Clasificación de Departamentos**
    Oficialmente, solo existen 22 entidades subnacionales de primer orden denominadas como departamentos en Guatemala. Sin embargo, al revisar la lista, se identifican 23 nombres de departamentos, exceptuando el nulo, lo que indica una clasificación errónea. Al examinar detenidamente, se encuentra que hay dos entradas: "Ciudad Capital" y "Guatemala", las cuales hacen referencia al mismo departamento. El nombre correcto es "Departamento de Guatemala"; por lo tanto, se procederá a unir estas dos entradas y renombrarlas adecuadamente.

Es importante destacar que se abordará la correcta identificación de valores nulos en esta columna en esta misma sección.

In [337]:
# Dictionary mapping incorrect department names to correct ones
department_corrections = {
    "Ciudad Capital": "Guatemala", # Corrected
    "nan": np.nan  # Replace nan with None or keep it as needed
}

In [338]:
# Replace department names in the DataFrame
df['department'] = df['department'].replace(department_corrections)

In [339]:
# Count occurrences of each unique value
value_counts = df['department'].value_counts()
# Display the occurrences
print(value_counts)

Guatemala         3041
Escuintla          628
San Marcos         574
Huehuetenango      516
Quetzaltenango     491
Suchitepequez      385
Alta Verapaz       374
Izabal             368
Peten              366
Chimaltenango      359
Sacatepequez       319
Retalhuleu         316
Jutiapa            310
Quiche             244
Chiquimula         170
Santa Rosa         158
Jalapa             151
Solola             138
El Progreso        125
Baja Verapaz       114
Zacapa              94
Totonicapan         90
Name: department, dtype: int64


#### **(2) Columna "municipality"**

Al igual que en la justificación presentada anteriormente, se abordarán los problemas de capitalización en la columna de municipios para garantizar la consistencia y la claridad en el análisis.

Además, a partir de las tablas de frecuencia, hemos identificado que hay varias entradas en esta columna que comienzan con la palabra "zona". Dado el contexto de los datos, se ha determinado que estas entradas son incorrectas y no reflejan adecuadamente la estructura geográfica de Guatemala. Por lo tanto, se procederá a clasificar todas estas entradas como "Guatemala" para asegurar que el dataset mantenga una representación precisa y uniforme de los municipios del país.

In [340]:
# Capitalizing the values of a single column
df['municipality'] = df['municipality'].str.title()
# Replace entries that start with "Zona"
df['municipality'] = df['municipality'].replace(to_replace=r'^Zona.*', value='Guatemala', regex=True)
# Replace 'nan' strings with np.nan
df['municipality'] = df['municipality'].replace('nan', np.nan)

In [341]:
# Count occurrences of each unique value
value_counts = df['municipality'].value_counts()
# Display the occurrences
print(value_counts)

Guatemala             1567
Mixco                  428
Villa Nueva            370
Quetzaltenango         248
Retalhuleu             185
                      ... 
San Miguel Siguila       1
Patzite                  1
Chiche                   1
Zunil                    1
San Jorge                1
Name: municipality, Length: 328, dtype: int64


**Observaciones 💡 -->**

> - Tras realizar una exhaustiva investigación, se ha confirmado que, salvo por algunas faltas de tildes, no existen discrepancias en los nombres de los municipios en el dataset. Esta afirmación se respalda con la información disponible en este [sitio web](https://es.wikipedia.org/wiki/Anexo:Municipios_de_Guatemala), donde se listan los departamentos junto con sus respectivos municipios.

#### **(3) Columna "address"**

In [342]:
# Count occurrences of each unique value
value_counts = df['address'].value_counts()
# Display the occurrences
print(value_counts)

CABECERA MUNICIPAL                                                       270
BARRIO EL CENTRO                                                          60
BARRIO EL CALVARIO                                                        19
BARRIO EL PORVENIR                                                        19
BARRIO LAS FLORES                                                         15
                                                                        ... 
SECTOR 1 MANZANA "B2 LOTE 31 Y 32, COLONIA PLANES DEL MILAGRO, ZONA 0      1
28 CALLE  28-00 COLONIA  EL  MOLINO                                        1
SECTOR 1, MANZANA  "B"  LOTES 31 Y 32, COLONIA PLANES DEL MILAGRO          1
CANTON RIO SALJÁ KM. 13 1/2 CHINAUTLA VIEJA                                1
BARRIO EL  CAMPO                                                           1
Name: address, Length: 6084, dtype: int64


##### **(1) Limpieza General de la Columna**

In [343]:
# Convert all entries to strings
df['address'] = df['address'].astype(str)
# Convert to lowercase
df['address'] = df['address'].str.lower()
# Remove leading/trailing whitespaces
df['address'] = df['address'].str.strip()
# Remove special characters and punctuation (keeping letters, numbers, spaces, and hyphens)
df['address'] = df['address'].str.replace(r'[^\w\s-]', '', regex=True)
# Remove extra spaces
df['address'] = df['address'].str.replace(r'\s+', ' ', regex=True)
# Remove accents
df['address'] = df['address'].apply(unidecode)
# Assign nan to empty strings and known invalid values
df['address'].replace(['', '--', '-', '---'], 'nan', inplace=True)

##### **(2) Manejo de Abreviaciones**

In [344]:
# Create a DataFrame from the abbreviation mapping
# ===== ===== IMPORTANT ===== =====
# This has been saved as a CSV file for easy access and use by others who may need it.
abbreviation_mapping = {
    'abbreviation': [
        'ave', 'ave.', 'km', 'av', 'cl.', 'z.'
    ],
    'full_form': [
        'avenida', 'avenida', 'kilometro', 'avenida', 'calle', 'zona'
    ]
}

abbreviation_df = pd.DataFrame(abbreviation_mapping)
# Save the DataFrame as a CSV file
abbreviation_df.to_csv('abbreviation_mapping.csv', index=False)

In [345]:
# Load the abbreviation mapping from CSV
abbreviation_df = pd.read_csv('abbreviation_mapping.csv')

# Create a dictionary from the DataFrame for easier access
abbreviation_mapping = dict(zip(abbreviation_df['abbreviation'], abbreviation_df['full_form']))

# Function to normalize abbreviations in addresses using the CSV mapping
def normalize_abbreviations(address):
    for abbr, full in abbreviation_mapping.items():
        address = re.sub(r'\b' + re.escape(abbr) + r'\b', full, address, flags=re.IGNORECASE)
    return address

In [346]:
# Apply normalization to the 'address' column
df['address'] = df['address'].apply(normalize_abbreviations)

##### **(3) Manejo de Errores Tipográficos**

In [347]:
# List of valid words (this can be extended)
valid_words = ['calle', 'zona', 'avenida']

# Function to check and correct spelling
def correct_spelling(address):
    words = address.split()
    corrected_words = []
    for word in words:
        # Find close matches from the valid words
        close_matches = difflib.get_close_matches(word, valid_words, n=1, cutoff=0.8)
        # If a close match is found, use it; otherwise, keep the original word
        corrected_word = close_matches[0] if close_matches else word
        corrected_words.append(corrected_word)
    return ' '.join(corrected_words)

In [348]:
# Apply spelling correction to the 'address' column
df['address'] = df['address'].apply(correct_spelling)

##### **(4) Levenshtein Distance Filtering Para Valores Duplicados**

<div class="alert alert-block alert-info">
<b>Nota:</b> La distancia de Levenshtein (LD) es una medida de la similitud entre dos cadenas de texto, la cadena fuente (s) y la cadena objetivo (t). La distancia es el número de eliminaciones, inserciones o sustituciones necesarias para transformar s en t. Cuanto mayor sea la distancia de Levenshtein, más diferentes serán las cadenas.
</div>

In [349]:
# Function to replace duplicates using fuzzy matching
def replace_duplicates_fuzzy(df, column, threshold=80):
    unique_addresses = df[column].unique()
    for index, address in enumerate(unique_addresses):
        # Find similar addresses in the DataFrame
        similar_addresses = process.extract(address, df[column], scorer=fuzz.token_sort_ratio)
        # Replace similar addresses with the current address if similarity exceeds the threshold
        for match in similar_addresses:
            if match[1] >= threshold:
                df[column] = df[column].replace(match[0], address)
    return df

In [350]:
# ===== ===== IMPORTANT ===== =====
# Only run this cell if absolutely necessary. 
# This operation can take approximately 20 minutes to complete.
# Apply the replacement of duplicates
df = replace_duplicates_fuzzy(df, 'address', threshold=90)

In [351]:
# ===== ===== IMPORTANT ===== =====
# For testing only.
result_df_copy = df.copy()

##### **(5) Estandarización de Direcciones**

In [352]:
# Function to standardize addresses
def standardize_address(address):
    # Ensure the address is a string
    if isinstance(address, str):
        match = re.search(r'(\d+[a-zA-Z]?)\s+(calle|avenida)\s+(\d+[a-zA-Z]?\s*-\s*\d+[a-zA-Z]?)?\s*(zona\s*\d+)?', address, re.IGNORECASE)
        if match:
            street_number = match.group(1).rstrip('ab')  # Remove suffixes like 'a', 'b'
            street_type = match.group(2).capitalize()  # Capitalize street type
            number = match.group(3) if match.group(3) else ''  # Get the street number part
            zone = match.group(4).strip() if match.group(4) else ''  # Get the zone
            if not zone: return np.nan
            # Capitalize "Zona" and create a standardized address
            standardized = f"{street_number} {street_type} {number.strip()} {zone.capitalize()}".strip()
            return standardized if standardized else np.nan
    # Return np.nan if no match is found
    return np.nan
# Apply the standardization function to the address column
df['address'] = df['address'].apply(standardize_address)

In [353]:
# Remove unwanted spaces
df['address'] = df['address'].str.replace(r'\s+', ' ', regex=True)  # Replace multiple spaces with a single space
df['address'] = df['address'].str.strip()  # Remove leading/trailing spaces

In [354]:
# Count occurrences of each unique value
value_counts = df['address'].value_counts()
# Display the occurrences
print(value_counts)

2 Avenida 2-80 Zona 1     51
2 Avenida 2-27 Zona 1     46
6 Avenida 1-47 Zona 1     39
1 Avenida 0-68 Zona 2     34
4 Calle 7-19 Zona 1       30
                          ..
10 Avenida 5-02 Zona 4     1
3 Calle 5-43 Zona 4        1
5 Calle 4-96 Zona 11       1
9 Avenida 1-60 Zona 5      1
9 Calle 15-26 Zona 1       1
Name: address, Length: 477, dtype: int64


In [355]:
# Replace string 'nan' with actual np.nan in the 'address' column only
df['address'].replace('nan', np.nan, inplace=True)

In [356]:
# Display modified DataFrame.
df.head(3)

,code,district,department,municipality,school_name,address,phone,supervisor,director,education_level,sector,area,status,teaching_mode,schedule,plan,departmental
0,16-01-0138-46,16-031,Alta Verapaz,Coban,COLEGIO COBAN,NaN,77945104,MERCEDES JOSEFINA TORRES GALVEZ,GUSTAVO ADOLFO SIERRA POP,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
1,16-01-0139-46,16-031,Alta Verapaz,Coban,COLEGIO PARTICULAR MIXTO VERAPAZ,NaN,77367402,MERCEDES JOSEFINA TORRES GALVEZ,GILMA DOLORES GUAY PAZ DE LEAL,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
2,16-01-0140-46,16-031,Alta Verapaz,Coban,"COLEGIO ""LA INMACULADA""",2 Avenida 2-27 Zona 1,78232301,MERCEDES JOSEFINA TORRES GALVEZ,VIRGINIA SOLANO SERRANO,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ


#### **(4) Columna "education_level"**

#### **(5) Columna "supervisor"**